In [1]:
import sys
import dotenv
import os
dotenv.load_dotenv()
sys.path.append(os.getenv('MAINDIR'))
from main import MoviesDatabase

synopsis = [movie.description for movie in MoviesDatabase.query.all()]

In [2]:
from rake_nltk import Rake
import nltk

r = Rake()
r.extract_keywords_from_text('Im so fuckin awesome!')
print(r.get_ranked_phrases_with_scores())


[(4.0, 'fuckin awesome'), (1.0, 'im')]


In [3]:
keywords = []

for syno in synopsis:
    r.extract_keywords_from_text(syno)
    keywords.append(r.get_ranked_phrases_with_scores()[:3])

In [4]:
import pandas as pd
df = pd.DataFrame({'keywords': keywords, 'synopsis': synopsis})

In [5]:
n = 11
df.iloc[n]['synopsis'], df.iloc[n].keywords

("Batman ventures into Gotham City's underworld when a sadistic killer leaves behind a trail of cryptic clues. As the evidence begins to lead closer to home and the scale of the perpetrator's plans become clear, he must forge new relationships, unmask the culprit and bring justice to the abuse of power and corruption that has long plagued the metropolis.",
 [(16.0, 'sadistic killer leaves behind'),
  (16.0, 'must forge new relationships'),
  (9.0, 'plans become clear')])

In [6]:
import spacy
from collections import Counter
from string import punctuation
nlp = spacy.load("en_core_web_sm")

def get_hotwords(text):
    result = []
    pos_tag = ['PROPN', 'ADJ', 'NOUN'] 
    doc = nlp(text.lower()) 
    for token in doc:
        if(token.text in nlp.Defaults.stop_words or token.text in punctuation):
            continue
        if(token.pos_ in pos_tag):
            result.append(token.text)
    return result

output = set(get_hotwords(df.iloc[n]['synopsis']))
most_common_list = Counter(output).most_common(10)
for item in most_common_list:
  print(item)

('relationships', 1)
('trail', 1)
('culprit', 1)
('corruption', 1)
('killer', 1)
('abuse', 1)
('scale', 1)
('city', 1)
('sadistic', 1)
('home', 1)


In [7]:
import yake

kw_extractor = yake.KeywordExtractor()
keywords = kw_extractor.extract_keywords(df.iloc[n]['synopsis'])
for kw in sorted(keywords, key=lambda x: x[1]):
  print(kw)



('Gotham City underworld', 0.0003156958886588181)
('sadistic killer leaves', 0.0010378185479078276)
('Gotham City', 0.0031146356952437464)
('ventures into Gotham', 0.00561955076575256)
('City underworld', 0.00561955076575256)
('Batman ventures', 0.007948096222458798)
('cryptic clues', 0.007948096222458798)
('sadistic killer', 0.010159422250580143)
('killer leaves', 0.010159422250580143)
('trail of cryptic', 0.010159422250580143)
('Gotham', 0.05572221155747183)
('City', 0.05572221155747183)
('plans become clear', 0.06815467895249562)
('forge new relationships', 0.06815467895249562)
('unmask the culprit', 0.06815467895249562)
('plagued the metropolis', 0.06815467895249562)
('Batman', 0.07862947958994491)
('clues', 0.07862947958994491)
('evidence begins', 0.08446611338711053)
('begins to lead', 0.08446611338711053)


In [8]:
from keybert import KeyBERT

kw_model = KeyBERT()
keywords = kw_model.extract_keywords(df.iloc[n]['synopsis'])
print(keywords)

/home/ksaff/miniconda3/envs/movie_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[('gotham', 0.5918), ('batman', 0.56), ('underworld', 0.3728), ('metropolis', 0.296), ('city', 0.286)]


In [9]:
keywords = kw_model.extract_keywords(synopsis, keyphrase_ngram_range=(1, 1), top_n=10)

In [10]:
print(keywords[0])

[('thor', 0.6371), ('hulk', 0.503), ('asgardian', 0.4379), ('hela', 0.3617), ('imprisoned', 0.3391), ('avenger', 0.3303), ('universe', 0.3156), ('gladiatorial', 0.2522), ('quest', 0.244), ('mighty', 0.2251)]


In [11]:
all_keywords = []

for keylist in keywords:
    only_words = [word[0] for word in keylist]
    all_keywords += only_words

In [12]:
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
 
ps = PorterStemmer()

stemmed_keywords = [ps.stem(word) for word in set(all_keywords)]
print(len(set(stemmed_keywords)))

16646


In [30]:
from stemming.porter2 import stem
from stemming.lovins import stem as lovins_stemmer
from nltk.stem import LancasterStemmer
from nltk.stem import WordNetLemmatizer
 
lemmatizer = WordNetLemmatizer()
nltk.download('wordnet')
lancaster = LancasterStemmer()

stemmed_keywords = [lemmatizer.lemmatize(word) for word in all_keywords]
print(len(set(stemmed_keywords)))

18826


[nltk_data] Downloading package wordnet to /home/ksaff/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [115]:
df = pd.read_csv('babynames.csv')
namelist = set(df['Name'])

no_names = [word for word in stemmed_keywords if word.capitalize() not in namelist]

In [173]:
value_counts=pd.Series(no_names).value_counts()
filtered_words = value_counts[value_counts > 2]

In [174]:
len(filtered_words)

4761

In [175]:
import numpy as np

words = np.array(filtered_words.keys())

In [176]:
from transformers import AutoTokenizer, AutoModel
from bert_transformer import BertTransformer

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = AutoModel.from_pretrained("bert-base-uncased").to('cuda:0')

bert_transformer = BertTransformer(tokenizer, model, 500)
vectorized = bert_transformer.fit_transform(words)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/home/ksaff/Desktop/Movie_Base_And_Recommendations/nlp_synopsis_exploration/bert_transformer.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(tokenized_text).unsqueeze(0).to(self.device),


In [223]:
import torch

tensor_1 = vectorized[0]
for i in range(len(vectorized[1:])):
    if torch.allclose(tensor_1, vectorized[i + 1], rtol=50):
        print(i + 1)

10
1541
4097


In [229]:
words[4097]

'creation'

In [193]:
from sklearn.cluster import KMeans
kmeans = KMeans(500).fit(vectorized)

In [204]:
data = {'Cluster': kmeans.labels_, 'Word': words}
df = pd.DataFrame(data)

In [205]:
df[df['Cluster'] == 101]

,Cluster,Word
310,101,road
322,101,farm
597,101,rural
829,101,building
1394,101,cell
1426,101,plant
1533,101,refuge
1916,101,feudal
2149,101,mill
2838,101,repair
